In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np


# Config

BATCH_SIZE = 32
IMG_SIZE = 224
EPOCHS_PHASE_A = 10   
EPOCHS_PHASE_B = 5    
LR_PHASE_A = 1e-3
LR_PHASE_B = 1e-4
DATA_ROOT = r"D:\dataset_split"
N_LAST_LAYERS = 20  

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


# Data Augmentation

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),   
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness = 0.2, contrast = 0.2, saturation = 0.2),
    transforms.ToTensor(),
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor()
])

train_dataset = datasets.ImageFolder(root = f"{DATA_ROOT}/train", transform = train_transform)
val_dataset   = datasets.ImageFolder(root = f"{DATA_ROOT}/val", transform = val_transform)
test_dataset  = datasets.ImageFolder(root = f"{DATA_ROOT}/test", transform = val_transform)

train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True)
val_loader   = DataLoader(val_dataset, batch_size = BATCH_SIZE, shuffle = False)
test_loader  = DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle = False)

num_classes = len(train_dataset.classes)
print("Classes:", train_dataset.classes)


# Focal Loss

class FocalLoss(nn.Module):
    def __init__(self, alpha = 1, gamma = 2, reduction = "mean"):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = nn.CrossEntropyLoss(reduction = "none")(inputs, targets)
        pt = torch.exp(-ce_loss) 
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss

        if self.reduction == "mean":
            return focal_loss.mean()
        elif self.reduction == "sum":
            return focal_loss.sum()
        else:
            return focal_loss

criterion = FocalLoss()


# Model: ResNet50

resnet = models.resnet50(pretrained = True)
resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)
model = resnet.to(device)


# Training Function

def train_model(model, criterion, optimizer, train_loader, val_loader, epochs, phase_name = "Phase"):
    for epoch in range(epochs):
        # Train
        model.train()
        running_loss, running_correct = 0.0, 0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * imgs.size(0)
            running_correct += (outputs.argmax(1) == labels).sum().item()

        train_loss = running_loss / len(train_loader.dataset)
        train_acc = running_correct / len(train_loader.dataset)

        # Validation
        model.eval()
        val_loss, val_correct = 0.0, 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * imgs.size(0)
                val_correct += (outputs.argmax(1) == labels).sum().item()

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct / len(val_loader.dataset)

        print(f"{phase_name} Epoch [{epoch+1}/{epochs}] "
              f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} "
              f"| Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
    return model


# Phase A: Train classifier

for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True

optimizer = optim.Adam(model.fc.parameters(), lr = LR_PHASE_A)
model = train_model(model, criterion, optimizer, train_loader, val_loader, 
                    epochs = EPOCHS_PHASE_A, phase_name = "Phase A")


# Phase B: Fine-tuning last N layers

for name, param in model.named_parameters():
    param.requires_grad = False

for name, param in list(model.named_parameters())[-N_LAST_LAYERS:]:
    param.requires_grad = True

optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr = LR_PHASE_B)
model = train_model(model, criterion, optimizer, train_loader, val_loader, 
                    epochs = EPOCHS_PHASE_B, phase_name = "Phase B")


# Test 

model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        preds = outputs.argmax(1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

test_acc = (all_preds == all_labels).sum() / len(all_labels)
print(f"Test Accuracy: {test_acc:.4f}")

cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:")
print(cm)

cr = classification_report(all_labels, all_preds, target_names=train_dataset.classes)
print("Classification Report:")
print(cr)


# Save model

torch.save(model.state_dict(), "D:/saved_models/resnet50_finetuned_focalloss.pt")


Device: cuda
Classes: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']


d:\Anaconda\envs\plant\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Anaconda\envs\plant\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Phase A Epoch [1/10] Train Loss: 0.5082 | Train Acc: 0.7629 | Val Loss: 0.2732 | Val Acc: 0.8392
Phase A Epoch [2/10] Train Loss: 0.2380 | Train Acc: 0.8595 | Val Loss: 0.1563 | Val Acc: 0.8931
Phase A Epoch [3/10] Train Loss: 0.2042 | Train Acc: 0.8714 | Val Loss: 0.1475 | Val Acc: 0.9038
Phase A Epoch [4/10] Train Loss: 0.1854 | Train Acc: 0.8838 | Val Loss: 0.2257 | Val Acc: 0.8518
Phase A Epoch [5/10] Train Loss: 0.1793 | Train Acc: 0.8833 | Val Loss: 0.1648 | Val Acc: 0.8950
Phase A Epoch [6/10] Train Loss: 0.1629 | Train Acc: 0.8917 | Val Loss: 0.1873 | Val Acc: 0.8727
Phase A Epoch [7/10] Train Loss: 0.1539 | Train Acc: 0.8974 | Val Loss: 0.1930 | Val Acc: 0.8829
Phase A Epoch [8/10] Train Loss: 0.1641 | Train Acc: 0.8935 | Val Loss: 0.1270 | Val Acc: 0.9096
Phase A Epoch [9/10] Train Loss: 0.1557 | Train Acc: 0.8968 | Val Loss: 0.1385 | Val Acc: 0.9091
Phase A Epoch [10/10] Train Loss: 0.1460 | Train Acc: 0.9026 | Val Loss: 0.2230 | Val Acc: 0.8649
Phase B Epoch [1/5] Train Los